In [4]:
!pip install -q kaggle


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [27]:
!kaggle datasets download -d maedemaftouni/large-covid19-ct-slice-dataset


Dataset URL: https://www.kaggle.com/datasets/maedemaftouni/large-covid19-ct-slice-dataset
License(s): other
 12%|████▉                                   | 260M/2.06G [00:01<00:08, 219MB/s]^C
 13%|█████▎                                  | 281M/2.06G [00:02<00:13, 138MB/s]
User cancelled operation


In [19]:
!pwd

/workspaces/Chest-Cancer-Classification


In [18]:
os.chdir("../")

In [14]:
from dataclasses import dataclass
from pathlib import Path

# Define a data class to hold configuration details for data ingestion
@dataclass(frozen=True)
class DataIngestionConfig:
    # The root directory where all data ingestion artifacts will be stored
    root_dir: Path
    
    # The URL or identifier of the data source (e.g., a Kaggle dataset URL or identifier)
    source_URL: str
    
    # The local file path where the downloaded data file will be saved
    local_data_file: Path
    
    # The directory where the data file will be unzipped or extracted
    unzip_dir: Path


In [15]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [16]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):
        
        # Initialize the ConfigurationManager with file paths for config and parameters
        # Read the configuration and parameters from the specified file paths
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        # Create the root directory specified in the configuration
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        # Extract the data ingestion configuration from the loaded config
        config = self.config.data_ingestion
        
        # Create the directory specified in the data ingestion config
        create_directories([config.root_dir])
        
        # Create and return a DataIngestionConfig object using the extracted data
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config

In [17]:
import os
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size
import subprocess

In [20]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self) -> str:
        """
        Download data from Kaggle using the Kaggle CLI.
        """
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs(self.config.root_dir, exist_ok=True)  # Ensure the root directory exists
            
            logger.info(f"Downloading data from Kaggle using dataset URL: {dataset_url}")

            # Using subprocess to call the Kaggle CLI command for dataset download
            command = f"kaggle datasets download -d maedemaftouni/large-covid19-ct-slice-dataset -p {self.config.root_dir}"
            subprocess.run(command, shell=True, check=True)

            # Move the downloaded file to the specified location
            if os.path.exists(zip_download_dir):
                logger.info(f"Downloaded data to {zip_download_dir}")
            else:
                raise FileNotFoundError(f"Failed to download dataset. File not found at {zip_download_dir}")

        except Exception as e:
            logger.error(f"An error occurred while downloading the file: {e}")
            raise e

    def extract_zip_file(self):
        """
        Extract the zip file into the data directory.
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        
        try:
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
            logger.info(f"Extracted zip file to {unzip_path}")
        except zipfile.BadZipFile as e:
            logger.error(f"Failed to extract zip file: {e}")
            raise e
        except FileNotFoundError as e:
            logger.error(f"Zip file not found: {e}")
            raise e


In [26]:
import shutil
import os

# Path to the directory where Kaggle was installed
kaggle_path = '/usr/local/python/3.10.13/lib/python3.10/site-packages/kaggle-1.6.17.dist-info'

# Remove the directory manually
if os.path.exists(kaggle_path):
    shutil.rmtree(kaggle_path, ignore_errors=True)


In [25]:
# Function to reinstall Kaggle CLI
def reinstall_kaggle():
    try:
        # Uninstall Kaggle CLI
        subprocess.run(["pip", "uninstall", "kaggle", "-y"], check=True)
        # Reinstall Kaggle CLI
        print ("a")
        subprocess.run(["pip", "install", "kaggle"], check=True)
        print("Kaggle CLI reinstalled successfully.")
    except subprocess.CalledProcessError as e:
        print(f"Error during Kaggle CLI installation: {e}")

# Function to run the data ingestion code
def run_data_ingestion():
    try:
        # Ensure Kaggle is installed
        subprocess.run(["kaggle", "--version"], check=True)
        # Import your classes
        from cnnClassifier import ConfigurationManager, DataIngestion
        
        # Configuration and data ingestion
        config = ConfigurationManager()
        data_ingestion_config = config.get_data_ingestion_config()
        data_ingestion = DataIngestion(config=data_ingestion_config)
        data_ingestion.download_file()
        data_ingestion.extract_zip_file()
    except Exception as e:
        print(f"An error occurred: {e}")
        # Reinstall Kaggle CLI if it fails
        reinstall_kaggle()
        # Retry running the data ingestion code
        print("x")
        run_data_ingestion()

# Run the data ingestion process
run_data_ingestion()

An error occurred: [Errno 2] No such file or directory: 'kaggle'


error: uninstall-no-record-file

× Cannot uninstall kaggle 1.6.17
╰─> The package's contents are unknown: no RECORD file was found for kaggle.

hint: You might be able to recover from this via: pip install --force-reinstall --no-deps kaggle==1.6.17


Found existing installation: kaggle 1.6.17
Error during Kaggle CLI installation: Command '['pip', 'uninstall', 'kaggle', '-y']' returned non-zero exit status 1.
x
An error occurred: [Errno 2] No such file or directory: 'kaggle'


error: uninstall-no-record-file

× Cannot uninstall kaggle 1.6.17
╰─> The package's contents are unknown: no RECORD file was found for kaggle.

hint: You might be able to recover from this via: pip install --force-reinstall --no-deps kaggle==1.6.17


Found existing installation: kaggle 1.6.17
Error during Kaggle CLI installation: Command '['pip', 'uninstall', 'kaggle', '-y']' returned non-zero exit status 1.
x
An error occurred: [Errno 2] No such file or directory: 'kaggle'


...

error: uninstall-no-record-file

× Cannot uninstall kaggle 1.6.17
╰─> The package's contents are unknown: no RECORD file was found for kaggle.

hint: You might be able to recover from this via: pip install --force-reinstall --no-deps kaggle==1.6.17


Found existing installation: kaggle 1.6.17
Error during Kaggle CLI installation: Command '['pip', 'uninstall', 'kaggle', '-y']' returned non-zero exit status 1.
x
An error occurred: [Errno 2] No such file or directory: 'kaggle'


error: uninstall-no-record-file

× Cannot uninstall kaggle 1.6.17
╰─> The package's contents are unknown: no RECORD file was found for kaggle.

hint: You might be able to recover from this via: pip install --force-reinstall --no-deps kaggle==1.6.17


Found existing installation: kaggle 1.6.17


KeyboardInterrupt: 